# Reading multiple files in Tensorflow 2
Author: [Biswajit Sahoo](https://biswajitsahoo1111.github.io/)

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/biswajitsahoo1111/cbm_codes_open/blob/master/notebooks/Reading_multiple_files_in_Tensorflow_2.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/biswajitsahoo1111/cbm_codes_open/blob/master/notebooks/Reading_multiple_files_in_Tensorflow_2.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://www.dropbox.com/s/o4aevvuqr39kq20/Reading_multiple_files_in_Tensorflow_2.ipynb?dl=1"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In this post, we will read multiple `.csv` files into Tensorflow using generators. But the method we will discuss is general enough to work for other file formats as well. We will demonstrate the procedure using 500 `.csv` files. These files have been created using random numbers. Each file contains only 1024 numbers in one column. This method can easily be extended to huge datasets involving thousands of `.csv` files. As the number of files becomes large, we can't load the whole data into memory. So we have to work with chunks of it. Generators help us do just that conveniently. In this post, we will read multiple files using a custom generator.
 
This post is self-sufficient in the sense that readers don't have to download any data from anywhere. Just run the following codes sequentially. First, a folder named "random_data" will be created in current working directory and `.csv` files will be saved in it. Subsequently files will be read from that folder and processed. Just make sure that your current working directory doesn't have an old folder named "random_data". Then run the following codes. Jupyter notebook of this post can be found [here](https://github.com/biswajitsahoo1111/cbm_codes_open/blob/master/notebooks/Reading_multiple_files_in_Tensorflow_2.ipynb).

We will use `Tensorflow 2` to run our deep learning model. `Tensorflow` is very flexible. A given task can be done in different ways in it. The method we will use is not the only one. Readers are encouraged to explore other ways of doing the same. Below is an outline of three different tasks considered in this post.

# Outline:
1. Create 500 `".csv"` files and save it in the folder "random_data" in current directory.
2. Write a generator that reads data from the folder in chunks and preprocesses it.
3. Feed the chunks of data to a CNN model and train it for several epochs.

## 1. Create 500 `.csv` files of random data

As we intend to train a CNN model for classification using our data, we will generate data for 5 different classes. Following is the process that we will follow.
* Each `.csv` file will have one column of data with 1024 entries.
* Each file will be saved using one of the following names (Fault_1, Fault_2, Fault_3, Fault_4, Fault_5). The dataset is balanced, meaning, for each category, we have approximately same number of observations. Data files in "Fault_1" 
category will have names as "Fault_1_001.csv", "Fault_1_002.csv", "Fault_1_003.csv", ..., "Fault_1_100.csv". Similarly for other classes.

In [1]:
import numpy as np
import os
import glob
np.random.seed(1111)

First create a function that will generate random files. 

In [2]:
def create_random_csv_files(fault_classes, number_of_files_in_each_class):
    os.mkdir("./random_data/")  # Make a directory to save created files.
    for fault_class in fault_classes:
        for i in range(number_of_files_in_each_class):
            data = np.random.rand(1024,)
            file_name = "./random_data/" + eval("fault_class") + "_" + "{0:03}".format(i+1) + ".csv" # This creates file_name
            np.savetxt(eval("file_name"), data, delimiter = ",", header = "V1", comments = "")
        print(str(eval("number_of_files_in_each_class")) + " " + eval("fault_class") + " files"  + " created.")

Now use the function to create 100 files each for five fault types. 

In [3]:
create_random_csv_files(["Fault_1", "Fault_2", "Fault_3", "Fault_4", "Fault_5"], number_of_files_in_each_class = 100)

100 Fault_1 files created.
100 Fault_2 files created.
100 Fault_3 files created.
100 Fault_4 files created.
100 Fault_5 files created.


In [4]:
files = glob.glob("./random_data/*")
print("Total number of files: ", len(files))
print("Showing first 10 files...")
files[:10]

Total number of files:  500
Showing first 10 files...


['./random_data\\Fault_1_001.csv',
 './random_data\\Fault_1_002.csv',
 './random_data\\Fault_1_003.csv',
 './random_data\\Fault_1_004.csv',
 './random_data\\Fault_1_005.csv',
 './random_data\\Fault_1_006.csv',
 './random_data\\Fault_1_007.csv',
 './random_data\\Fault_1_008.csv',
 './random_data\\Fault_1_009.csv',
 './random_data\\Fault_1_010.csv']

To extract labels from file name, extract the part of the file name that corresponds to fault type. 

In [5]:
print(files[0])

./random_data\Fault_1_001.csv


In [6]:
print(files[0][14:21])

Fault_1


Now that data have been created, we will go to the next step. That is, define a generator, preprocess the time series like data into a matrix like shape such that a 2-D CNN can ingest it. 

## 2. Write a generator that reads data in chunks and preprocesses it
Generator are similar to functions with one important difference. While functions produce all their outputs at once, generators produce their outputs one by one and that too when asked. `yield` keyword converts a function into a generator. Generators can run for a fixed number of times or indefinitely depending on the loop structure used inside it. For our application, we will use a generator that runs indefinitely.

The following generator takes a list of file names as first argument. The second argument is `batch_size`. `batch_size` determines how many files we will process at one go. This is determined by how much memory do we have. If all data can be loaded into memory, there is no need for generators. In case our data size is huge, we can process chunks of it.  

As we will be solving a classification problem, we have to assign labels to each raw data. We will use following labels for convenience.

|Class| Label|
|-----|------|
|Fault_1| 0|
|Fault_2| 1|
|Fault_3| 2|
|Fault_4| 3|
|Fault_5| 4|

The generator will `yield` both data and labels.

In [7]:
import pandas as pd
import re            # To match regular expression for extracting labels

In [8]:
def data_generator(file_list, batch_size = 20):
    i = 0
    while True:
        if i*batch_size >= len(file_list):  # This loop is used to run the generator indefinitely.
            i = 0
            np.random.shuffle(file_list)
        else:
            file_chunk = file_list[i*batch_size:(i+1)*batch_size] 
            data = []
            labels = []
            label_classes = ["Fault_1", "Fault_2", "Fault_3", "Fault_4", "Fault_5"]
            for file in file_chunk:
                temp = pd.read_csv(open(file,'r')) # Change this line to read any other type of file
                data.append(temp.values.reshape(32,32,1)) # Convert column data to matrix like data with one channel
                pattern = "^" + eval("file[14:21]")      # Pattern extracted from file_name
                for j in range(len(label_classes)):
                    if re.match(pattern, label_classes[j]): # Pattern is matched against different label_classes
                        labels.append(j)  
            data = np.asarray(data).reshape(-1,32,32,1)
            labels = np.asarray(labels)
            yield data, labels
            i = i + 1

To read any other file format, inside the generator change the line that reads files. This will enable us to read different file formats, be it `.txt` or `.npz` or any other. Preprocessing of data, different from what we have done in this blog, can be done within the generator loop.

Now we will check whether the generator works as intended or not. We will set `batch_size` to 10. This means that files in chunks of 10 will be read and processed. The list of files from which 10 are chosen can be an ordered file list or shuffled list. In case, the files are not shuffled, use `np.random.shuffle(file_list)` to shuffle files. 

In the demonstration, we will read files from an ordered list. This will help us check any errors in the code. 

In [9]:
generated_data = data_generator(files, batch_size = 10)

In [10]:
num = 0
for data, labels in generated_data:
    print(data.shape, labels.shape)
    print(labels, "<--Labels")  # Just to see the lables
    print()
    num = num + 1
    if num > 5: break

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels



Run the above cell multiple times to observe different labels. Label 1 appears only when all the files corresponding to "Fault_1" have been read. There are 100 files for "Fault_1" and we have set batch_size to 10. In the above cell we are iterating over the generator only 6 times. When number of iterations become greater than 10, we see label 1 and subsequently other labels. This will happen only if our initial file list is not shuffled. If the original list is shuffled, we will get random labels.

Now we will create a `tensorflow dataset` using the generator. `Tensorflow` datasets can conveniently be used to train `tensorflow` models.

A `tensorflow dataset` can be created form numpy arrays or from generators.Here, we will create it using a generator. Use of the previously created generator as it is in `tensorflow datasets` doesn't work (Readers can verify this). This happens because of the inability of regular expression to compare a "string" with a "byte string". "byte strings" are generated by default in tensorflow. As a way around, we will make modifications to the earlier generator and use it with tensorflow datasets. Note that we will only modified three lines. Modified lines are accompanied by commented texts beside it.

In [11]:
import tensorflow as tf

In [12]:
def tf_data_generator(file_list, batch_size = 20):
    i = 0
    while True:
        if i*batch_size >= len(file_list):  
            i = 0
            np.random.shuffle(file_list)
        else:
            file_chunk = file_list[i*batch_size:(i+1)*batch_size] 
            data = []
            labels = []
            label_classes = tf.constant(["Fault_1", "Fault_2", "Fault_3", "Fault_4", "Fault_5"]) # This line has changed.
            for file in file_chunk:
                temp = pd.read_csv(open(file,'r'))
                data.append(temp.values.reshape(32,32,1)) 
                pattern = tf.constant(eval("file[14:21]"))  # This line has changed
                for j in range(len(label_classes)):
                    if re.match(pattern.numpy(), label_classes[j].numpy()):  # This line has changed.
                        labels.append(j)
            data = np.asarray(data).reshape(-1,32,32,1)
            labels = np.asarray(labels)
            yield data, labels
            i = i + 1

Test whether modified generator works or not.

In [13]:
check_data = tf_data_generator(files, batch_size = 10)

In [14]:
num = 0
for data, labels in check_data:
    print(data.shape, labels.shape)
    print(labels, "<--Labels")
    print()
    num = num + 1
    if num > 5: break

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels



Note that the new generator created by using a few `tensorflow` commands works just fine as our previous generator. This new generator can now be integrated with a `tensorflow dataset`.

In [15]:
batch_size = 15
dataset = tf.data.Dataset.from_generator(tf_data_generator,args= [files, batch_size],output_types = (tf.float32, tf.float32),
                                                output_shapes = ((None,32,32,1),(None,)))

Check whether `dataset` works or not.

In [16]:
num = 0
for data, labels in dataset:
    print(data.shape, labels.shape)
    print(labels)
    print()
    num = num + 1
    if num > 7: break

(15, 32, 32, 1) (15,)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(15,), dtype=float32)

(15, 32, 32, 1) (15,)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(15,), dtype=float32)

(15, 32, 32, 1) (15,)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(15,), dtype=float32)

(15, 32, 32, 1) (15,)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(15,), dtype=float32)

(15, 32, 32, 1) (15,)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(15,), dtype=float32)

(15, 32, 32, 1) (15,)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(15,), dtype=float32)

(15, 32, 32, 1) (15,)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1.], shape=(15,), dtype=float32)

(15, 32, 32, 1) (15,)
tf.Tensor([1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.], shape=(15,), dtype=float32)



This also works fine. Now, we will train a full CNN model using the generator. As is done in every model, we will first shuffle data files. Split the files into train, validation, and test set. Using the `tf_data_generator` create three tensorflow datasets corresponding to train, validation, and test data respectively. Finally, we will create a simple CNN model. Train it using train dataset, see its performance on validation dataset, and obtain prediction using test dataset. Keep in mind that our aim is not to improve performance of the model. As the data are random, don't expect to see good performance. The aim is only to create a pipeline. 

## 3. Building data pipeline and training CNN model

Before building the data pipeline, we will first move files corresponding to each fault class into different folders. This will make it convenient to split data into training, validation, and test set, keeping the balanced nature of the dataset intact.

In [17]:
import shutil

Create five different folders.

In [18]:
fault_folders = ["Fault_1", "Fault_2", "Fault_3", "Fault_4", "Fault_5"]
for folder_name in fault_folders:
    os.mkdir(os.path.join("./random_data", folder_name))

Move files into those folders.

In [19]:
for file in files:
    pattern = "^" + eval("file[14:21]")
    for j in range(len(fault_folders)):
        if re.match(pattern, fault_folders[j]):
            dest = os.path.join("./random_data/",eval("fault_folders[j]"))
            shutil.move(file, dest)

In [20]:
glob.glob("./random_data/*")

['./random_data\\Fault_1',
 './random_data\\Fault_2',
 './random_data\\Fault_3',
 './random_data\\Fault_4',
 './random_data\\Fault_5']

In [21]:
glob.glob("./random_data/Fault_1/*")[:10] # Showing first 10 files of Fault_1 folder

['./random_data/Fault_1\\Fault_1_001.csv',
 './random_data/Fault_1\\Fault_1_002.csv',
 './random_data/Fault_1\\Fault_1_003.csv',
 './random_data/Fault_1\\Fault_1_004.csv',
 './random_data/Fault_1\\Fault_1_005.csv',
 './random_data/Fault_1\\Fault_1_006.csv',
 './random_data/Fault_1\\Fault_1_007.csv',
 './random_data/Fault_1\\Fault_1_008.csv',
 './random_data/Fault_1\\Fault_1_009.csv',
 './random_data/Fault_1\\Fault_1_010.csv']

In [22]:
glob.glob("./random_data/Fault_3/*")[:10] # Showing first 10 files of Falut_3 folder

['./random_data/Fault_3\\Fault_3_001.csv',
 './random_data/Fault_3\\Fault_3_002.csv',
 './random_data/Fault_3\\Fault_3_003.csv',
 './random_data/Fault_3\\Fault_3_004.csv',
 './random_data/Fault_3\\Fault_3_005.csv',
 './random_data/Fault_3\\Fault_3_006.csv',
 './random_data/Fault_3\\Fault_3_007.csv',
 './random_data/Fault_3\\Fault_3_008.csv',
 './random_data/Fault_3\\Fault_3_009.csv',
 './random_data/Fault_3\\Fault_3_010.csv']

Prepare that data for training set, validation set, and test_set. For each fault type, we will keep 70 files for training, 10 files for validation and 20 files for testing.

In [23]:
fault_1_files = glob.glob("./random_data/Fault_1/*")
fault_2_files = glob.glob("./random_data/Fault_2/*")
fault_3_files = glob.glob("./random_data/Fault_3/*")
fault_4_files = glob.glob("./random_data/Fault_4/*")
fault_5_files = glob.glob("./random_data/Fault_5/*")

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
fault_1_train, fault_1_test = train_test_split(fault_1_files, test_size = 20, random_state = 5)
fault_2_train, fault_2_test = train_test_split(fault_2_files, test_size = 20, random_state = 54)
fault_3_train, fault_3_test = train_test_split(fault_3_files, test_size = 20, random_state = 543)
fault_4_train, fault_4_test = train_test_split(fault_4_files, test_size = 20, random_state = 5432)
fault_5_train, fault_5_test = train_test_split(fault_5_files, test_size = 20, random_state = 54321)

In [26]:
fault_1_train, fault_1_val = train_test_split(fault_1_train, test_size = 10, random_state = 1)
fault_2_train, fault_2_val = train_test_split(fault_2_train, test_size = 10, random_state = 12)
fault_3_train, fault_3_val = train_test_split(fault_3_train, test_size = 10, random_state = 123)
fault_4_train, fault_4_val = train_test_split(fault_4_train, test_size = 10, random_state = 1234)
fault_5_train, fault_5_val = train_test_split(fault_5_train, test_size = 10, random_state = 12345)

In [27]:
train_file_names = fault_1_train + fault_2_train + fault_3_train + fault_4_train + fault_5_train
validation_file_names = fault_1_val + fault_2_val + fault_3_val + fault_4_val + fault_5_val
test_file_names = fault_1_test + fault_2_test + fault_3_test + fault_4_test + fault_5_test

In [28]:
print("Number of train_files:" ,len(train_file_names))
print("Number of validation_files:" ,len(validation_file_names))
print("Number of test_files:" ,len(test_file_names))

Number of train_files: 350
Number of validation_files: 50
Number of test_files: 100


In [29]:
batch_size = 10
train_dataset = tf.data.Dataset.from_generator(tf_data_generator, args = [train_file_names, batch_size], 
                                              output_shapes = ((None,32,32,1),(None,)),
                                              output_types = (tf.float32, tf.float32))

validation_dataset = tf.data.Dataset.from_generator(tf_data_generator, args = [validation_file_names, batch_size],
                                                   output_shapes = ((None,32,32,1),(None,)),
                                                   output_types = (tf.float32, tf.float32))

test_dataset = tf.data.Dataset.from_generator(tf_data_generator, args = [test_file_names, batch_size],
                                             output_shapes = ((None,32,32,1),(None,)),
                                             output_types = (tf.float32, tf.float32))

Now create the model.

In [30]:
from tensorflow.keras import layers

In [31]:
model = tf.keras.Sequential([
    layers.Conv2D(16, 3, activation = "relu", input_shape = (32,32,1)),
    layers.MaxPool2D(2),
    layers.Conv2D(32, 3, activation = "relu"),
    layers.MaxPool2D(2),
    layers.Flatten(),
    layers.Dense(16, activation = "relu"),
    layers.Dense(5, activation = "softmax")
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0         
_________________________________________________________________
dense (Dense)                (None, 16)                18448     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 8

Compile the model.

In [32]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

Before we fit the model, we have to do one important calculation. Remember that our generators are infinite loops. So if no stopping criteria is given, it will run indefinitely. But we want our model to run for, say, 10 epochs. So our generator should loop over the data files just 10 times and no more. This is achieved by setting the arguments `steps_per_epoch` and `validation_steps` to desired numbers in `model.fit()`. Similarly while evaluating model, we need to set the argument `steps` to a desired number in `model.evaluate()`.

There are 350 files in training set. Batch_size is 10. So if the generator runs 35 times, it will correspond to one epoch. Therefor, we should set `steps_per_epoch` to 35. Similarly, `validation_steps = 5` and in `model.evaluate()`, `steps = 10`.

In [33]:
steps_per_epoch = np.int(np.ceil(len(train_file_names)/batch_size))
validation_steps = np.int(np.ceil(len(validation_file_names)/batch_size))
steps = np.int(np.ceil(len(test_file_names)/batch_size))
print("steps_per_epoch = ", steps_per_epoch)
print("validation_steps = ", validation_steps)
print("steps = ", steps)

steps_per_epoch =  35
validation_steps =  5
steps =  10


In [34]:
model.fit(train_dataset, validation_data = validation_dataset, steps_per_epoch = steps_per_epoch,
         validation_steps = validation_steps, epochs = 10)

Train for 35 steps, validate for 5 steps
Epoch 1/10
35/35 [==============================] - 8s 237ms/step - loss: 1.9128 - accuracy: 0.1771 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 2/10
35/35 [==============================] - 6s 168ms/step - loss: 1.6096 - accuracy: 0.1943 - val_loss: 1.6095 - val_accuracy: 0.2000
Epoch 3/10
35/35 [==============================] - 6s 170ms/step - loss: 1.6096 - accuracy: 0.2000 - val_loss: 1.6095 - val_accuracy: 0.2000
Epoch 4/10
35/35 [==============================] - 6s 170ms/step - loss: 1.6096 - accuracy: 0.2000 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 5/10
35/35 [==============================] - 6s 171ms/step - loss: 1.6111 - accuracy: 0.1943 - val_loss: 1.6093 - val_accuracy: 0.2000
Epoch 6/10
35/35 [==============================] - 6s 164ms/step - loss: 1.6096 - accuracy: 0.2029 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 7/10
35/35 [==============================] - 6s 167ms/step - loss: 1.6097 - accuracy: 0.2000 - val_l

In [35]:
test_loss, test_accuracy = model.evaluate(test_dataset, steps = 10)

10/10 [==============================] - 1s 129ms/step - loss: 1.6097 - accuracy: 0.1900


In [36]:
print("Test loss: ", test_loss)
print("Test accuracy:", test_accuracy)

Test loss:  1.6096720457077027
Test accuracy: 0.19


As expected, model performs terribly. 

## How to make predictions?
Until now, we have evaluated our model on a kept out test set. For our test set, both data and labels were known. So we evaluated its performance. But often times, for test set, we don't have access to true labels. Rather we have to make predictions on the data available. This is the case in online competitions where we have to submit our predictions on a test set for which don't know the labels. We will call this set (without any labels) the prediction set. This naming convention is arbitray but we will stick with it.

If the whole of our prediction set fits into memory, we can just call `model.predict()` on this data and then use `np.argmax()` to obtain predicted class labels. Otherwise, we can read files in prediction set in chunks, make predictions on the chunks and finally append our result.

Yet another pedantic way of doing this is to write a generator to read files from the prediciton set in chunks and make predictions on it. We will show how this approach works. As we don't have a prediction set yet, we will first create some files and save it to the prediction set.

In [37]:
def create_prediction_set(num_files = 20):
    os.mkdir("./random_data/prediction_set")
    for i in range(num_files):
        data = np.random.randn(1024,)
        file_name = "./random_data/prediction_set/"  + "file_" + "{0:03}".format(i+1) + ".csv" # This creates file_name
        np.savetxt(eval("file_name"), data, delimiter = ",", header = "V1", comments = "")
    print(str(eval("num_files")) + " "+ " files created in prediction set.")

Create some files for prediction set.

In [38]:
create_prediction_set(num_files = 55)

55  files created in prediction set.


In [39]:
prediction_files = glob.glob("./random_data/prediction_set/*")
print("Total number of files: ", len(prediction_files))
print("Showing first 10 files...")
prediction_files[:10]

Total number of files:  55
Showing first 10 files...


['./random_data/prediction_set\\file_001.csv',
 './random_data/prediction_set\\file_002.csv',
 './random_data/prediction_set\\file_003.csv',
 './random_data/prediction_set\\file_004.csv',
 './random_data/prediction_set\\file_005.csv',
 './random_data/prediction_set\\file_006.csv',
 './random_data/prediction_set\\file_007.csv',
 './random_data/prediction_set\\file_008.csv',
 './random_data/prediction_set\\file_009.csv',
 './random_data/prediction_set\\file_010.csv']

Now, we will create a generator to read these files in chunks. This generator will be slightly different from our previous generator. Firstly, we don't want the generator to run indefinitely. Secondly, we don't have any labels. So this generator should only `yield` data. This is how we achieve that.

In [40]:
def generator_for_prediction(file_list, batch_size = 20):
    i = 0
    while i <= (len(file_list)/batch_size):
        if i == np.floor(len(file_list)/batch_size):
            file_chunk = file_list[i*batch_size:len(file_list)]
            if len(file_chunk)==0:
                break
        else:
            file_chunk = file_list[i*batch_size:(i+1)*batch_size] 
        data = []
        for file in file_chunk:
            temp = pd.read_csv(open(file,'r'))
            data.append(temp.values.reshape(32,32,1)) 
        data = np.asarray(data).reshape(-1,32,32,1)
        yield data
        i = i + 1

Check whether the generator works or not.

In [41]:
pred_gen = generator_for_prediction(prediction_files,  batch_size = 10)
for data in pred_gen:
    print(data.shape)

(10, 32, 32, 1)
(10, 32, 32, 1)
(10, 32, 32, 1)
(10, 32, 32, 1)
(10, 32, 32, 1)
(5, 32, 32, 1)


Create a `tensorflow dataset`.

In [42]:
batch_size = 10
prediction_dataset = tf.data.Dataset.from_generator(generator_for_prediction,args=[prediction_files, batch_size],
                                                 output_shapes=(None,32,32,1), output_types=(tf.float32))

In [43]:
steps = np.int(np.ceil(len(prediction_files)/batch_size))
predictions = model.predict(prediction_dataset,steps = steps)

In [44]:
print("Shape of prediction array: ", predictions.shape)
predictions

Shape of prediction array:  (55, 5)


array([[0.21503152, 0.1857667 , 0.19835523, 0.2200414 , 0.18080522],
       [0.21068904, 0.18612576, 0.20154041, 0.21330442, 0.18834037],
       [0.20435034, 0.20236893, 0.2097578 , 0.20269258, 0.1808304 ],
       [0.222747  , 0.18716884, 0.19043802, 0.22575077, 0.1738954 ],
       [0.2254422 , 0.19780202, 0.18614103, 0.20861967, 0.18199512],
       [0.20128603, 0.17159882, 0.20370749, 0.25313848, 0.17026915],
       [0.22571482, 0.20458302, 0.1904429 , 0.19584061, 0.18341863],
       [0.2144547 , 0.184947  , 0.18816288, 0.22664   , 0.18579544],
       [0.211543  , 0.18050838, 0.21344864, 0.2154084 , 0.17909157],
       [0.2093142 , 0.18945844, 0.2079513 , 0.21437332, 0.17890278],
       [0.18830046, 0.1859439 , 0.20227744, 0.24265009, 0.1808281 ],
       [0.20933314, 0.18911888, 0.20666867, 0.21769525, 0.17718406],
       [0.23392989, 0.17569026, 0.19360954, 0.21315034, 0.18361998],
       [0.2202179 , 0.18947901, 0.2063513 , 0.20865704, 0.17529482],
       [0.2069124 , 0.19218966, 0.

Outsputs of prediction are 5 dimensional vector. This is so because we have used 5 neurons in the output layer and our activation function is softmax. The 5 dimensional output vector for an input add to 1. So it can be interpreted as probability. Thus we should classify the input to a class, for which prediction probability is maximum. To get the class corresponding to maximum probability, we can use `np.argmax()` command.

In [45]:
np.argmax(predictions, axis = 1)

array([3, 3, 2, 3, 0, 3, 0, 3, 3, 3, 3, 3, 0, 0, 2, 0, 3, 3, 0, 0, 3, 2,
       3, 3, 0, 3, 3, 3, 0, 0, 0, 2, 0, 3, 0, 3, 3, 3, 3, 0, 0, 0, 0, 3,
       3, 3, 3, 0, 0, 3, 0, 0, 0, 3, 3], dtype=int64)

The data are randomly generated. So we should not be surprised by this result. Also note that the for each new data, softmax outputs are close to each other. This means that the network is not too sure about the classification result. Softmax values for "Fault_2" are only marginally better than others. Therefore, all the data are classified as "Fault_2" type. 

This brings us to the end of the blog. As we had planned in the beginning, we have created random data files, a generator, and trained a model using that generator. The above code can be tweaked slightly to read any type of files other than `.csv`. And now we can train our model without worrying about the data size. Whether the data size is 10GB or 750GB, our approach will work for both.

As a final note, I want to stress that, this is not the only approach to do the task. As I have mentioned previously, in `Tensorflow`, you can do the same thing in several different ways. The approach I have chosen seemed natural to me. I have neither strived for efficiency nor elegance. If readers have any better idea, I would be happy to know of it.

I hope, this blog will be of help to readers. Please bring any errors or omissions to my notice.

**Update**: If along with reading, one has to perform complex transformations on extracted data (say, doing spectrogram on each segment of data, etc.), the naive approach presented in this blog may turn out to be slow. But there are ways to make these computations faster. The author himself achieved 10x speedup as compared to naive implementation in one application. If you run into this problem and are unable to solve it, contact the author. A blog discussing this speedup technique will be posted later.

Last modified: 27th April, 2020.